<center>
<img src="../../img/ods_stickers.jpg">
## Open Machine Learning Course
<center>
Author: Yury Kashnitsky, Data Scientist at Mail.Ru Group

This material is subject to the terms and conditions of the license [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Free use is permitted for any non-comercial purpose with an obligatory indication of the names of the authors and of the source.

## <center>Assignment #6. Part 1
### <center> Beating benchmarks in "Catch Me If You Can: Intruder Detection through Webpage Session Tracking"
    
[Competition](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2). The task is to beat "Assignment 6 baseline".

In [223]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [224]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

Reading original data

In [225]:
# Read the training and test data sets

PATH_TO_DATA = ('')
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id')

In [439]:
# Switch time1, ..., time10 columns to datetime type
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# Sort the data by time
train_df = train_df.sort_values(by='time1')

# Look at the first rows of the training set
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,...,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,1970-01-01 00:00:00,...,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22,0


Separate target feature 

In [440]:
y = train_df['target']

In [441]:
# Change site1, ..., site10 columns type to integer and fill NA-values with zeros
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# Load websites dictionary
with open(r"site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# Create dataframe for the dictionary
sites_dict = pd.DataFrame(list(site_dict.keys()), index=list(site_dict.values()), columns=['site'])
print(u'Websites total:', sites_dict.shape[0])
sites_dict.head()

Websites total: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


In [656]:
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].fillna(0).astype('datetime64[ns]')
test_df[times] = test_df[times].fillna(0).astype('datetime64[ns]')

In [657]:
# Our target variable
y_train = train_df['target']

# United dataframe of the initial data 
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# Index to split the training and test data sets
idx_split = train_df.shape[0]

full_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,945,2013-01-12 08:50:16,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,946,2013-01-12 08:50:18,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,952,2013-01-12 08:50:21,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22


In [659]:
# Dataframe with indices of visited websites in session
sites = ['site%s' % i for i in range(1, 11)]
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


In [660]:
# делаем список из топ10 сайтов, котрые Алиса посещает чаще всего! Это может быть очень жирным признаком
from collections import Counter
df_target = train_df[train_df['target']==1]
top10_list = []
for x,y in Counter(df_target[sites].values.flatten()).most_common(10):
    top10_list.append(x)


In [661]:
top10_list

[77, 80, 76, 29, 21, 81, 22, 879, 75, 82]

In [625]:
# последовательность с индексами
#sites_flatten = full_sites.values.flatten()

# искомая матрица
#full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                #sites_flatten,
                               #range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]


 Build Tf-Idf features based on sites. You can use `ngram_range`=(1, 3) and `max_features`=100000 or more

In [662]:
sites = ['site%s' % i for i in range(1, 11)]
full_df['sites_as_text'] = full_df[sites].astype(str).apply(' '.join,axis = 1)

In [663]:
full_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,sites_as_text
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,...,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,56 55 0 0 0 0 0 0 0 0
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,1970-01-01 00:00:00,...,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,0,1970-01-01 00:00:00,56 55 56 55 0 0 0 0 0 0
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17,946 946 951 946 946 945 948 784 949 946
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20,945 948 949 948 945 946 947 945 946 946
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22,947 950 948 947 950 952 946 951 946 947


In [678]:
#tmp_full_df = full_df
#vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=100000).fit(tmp_full_df['sites_as_text'])
#vectorized_sparsed_sites = vectorizer.transform(tmp_full_df['sites_as_text'])

In [668]:
#vectorizer = CountVectorizer(ngram_range=(1,1)) 
#vectorized_sparsed_sites = vectorizer.fit_transform(tmp_full_df['sites_as_text'])

Add features based on the session start time: hour, whether it's morning, day or night and so on.

говорят хватит бинарных ранее утро, утро, вечер, обед и воскресенье

In [596]:
%%time
new_full_df = full_df 

for i in range(1, 11):
    new_full_df['hour']          = new_full_df['time1'].apply(lambda time: pd.Timestamp(time).hour)
    new_full_df['early_morning'] = new_full_df['time1'].apply(lambda time: 1 if pd.Timestamp(time).hour < 10 else 0)
    new_full_df['morning']       = new_full_df['time1'].apply(lambda time: 1 if (pd.Timestamp(time).hour >= 10) and (pd.Timestamp(time).hour < 12) else 0)
    new_full_df['lunch']         = new_full_df['time1'].apply(lambda time: 1 if (pd.Timestamp(time).hour >= 12) and (pd.Timestamp(time).hour < 16) else 0)
    new_full_df['evening']       = new_full_df['time1'].apply(lambda time: 1 if (pd.Timestamp(time).hour >= 16) and (pd.Timestamp(time).hour < 24) else 0)
    new_full_df['sunday']        = new_full_df['time1'].apply(lambda time: 1 if pd.Timestamp(time).dayofweek == 6 else 0)
    new_full_df['top_count']     = new_full_df['time1'].apply(lambda time: 1 if pd.Timestamp(time).dayofweek == 6 else 0)
    

Wall time: 1min 33s


In [597]:
for i in range(1, 11):
    new_full_df.drop('time%s' % i, axis = 1, inplace=True)
    new_full_df.drop('site%s' % i, axis = 1, inplace=True)
new_full_df.drop('sites_as_text', axis = 1, inplace=True)  

In [598]:
scaler = StandardScaler(copy=False, with_mean=False)
scaled_new_df = pd.DataFrame(scaler.fit_transform(new_full_df))

In [139]:
#for i in range(1, 11):    
    #new_full_df.drop('site%s' % i, axis = 1, inplace=True)

In [609]:
result_matrix = hstack([scaled_new_df.values, full_sites_sparse_top10]).tocsr()

In [610]:
X_test = result_matrix[idx_split:, :]
X_train = result_matrix[:idx_split, :]
y_train = train_df['target']

In [621]:
X_test.shape

(82797, 16)

In [620]:
X_train.toarray()

array([[ 2.54615992,  2.4237465 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 2.54615992,  2.4237465 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 2.54615992,  2.4237465 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 7.32020976,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 7.32020976,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 7.32020976,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [612]:
X_test.shape

(82797, 16)

Perform cross-validation with logistic regression.

In [613]:
%%time
tss = TimeSeriesSplit(n_splits=7)
logit_cv = LogisticRegressionCV(Cs = [1], n_jobs=-1, scoring ='roc_auc', cv=tss)
#logit_cv = LogisticRegressionCV(Cs = [1], n_jobs=-1, scoring ='roc_auc', cv=tss)
logit_cv.fit(X_train, y_train)

Wall time: 8.61 s


In [614]:
logit_cv.scores_

{1: array([[ 0.76990733],
        [ 0.88200637],
        [ 0.82556784],
        [ 0.92876488],
        [ 0.68573551],
        [ 0.94360178],
        [ 0.91519554]])}

In [413]:
0.97182863

0.97182863

In [408]:
y_pred = logit_cv.predict_proba(X_test)[:, 1]
y_pred.shape

(82797,)

Make prediction for the test set and form a submission file.

In [410]:
test_pred = logit_cv.predict_proba(X_test)[:, 1]

In [411]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)


In [412]:
write_to_submission_file(test_pred, "assignment6_alice_submission.csv")